# Лабораторная работа №3
Классификация пингвинов
Выполнили студенты Зимин Андрей Валерьевич, Жилин Андрей Игроевич

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import weighted

from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import recall_score as rec
from sklearn.metrics import precision_score as pre
from sklearn.model_selection import train_test_split
from matplotlib.ticker import MaxNLocator
import qrcode
import numpy as np
from matplotlib import pyplot as plt
import random
import string
from typing import Callable, List, Union
from numpy.typing import NDArray
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import recall_score as rec
from sklearn.metrics import precision_score as pre
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from matplotlib.ticker import MaxNLocator

## Задание 1
1. Загрузить данные из файла «pinguins.csv». Проверить, что загружены все 333 кортежа данных по всем 7 признакам  
2. Определить типы данных
3. Определить параметры числовых данных
4. Для нечисловых (текстовых) данных определить количество записей по каждому элементу и визуализировать гистограммы, например, так (для признака «species»)
5. Для каждого текстового признака построить мозаику 2D диаграмм рассеяния выбирая в качестве параметров всевозможные пары числовых признаков, аналогично мозаике диаграмм для ирисов.

### 1. Загрузить данные из файла «pinguins.csv». Проверить, что загружены все 333 кортежа данных по всем 7 признакам

In [ ]:
df = pd.read_csv("./data/penguins.csv")
df

Загрузили датасет. Действительно 7 признаков и 333 строки

### 2. Определить типы данных

In [ ]:
df.info()

species            - категориальный признак  
island             - категориальный признак  
bill_length_mm     - числовой признак  
bill_depth_mm      - числовой признак  
flipper_length_mm  - числовой признак  
body_mass_g        - числовой признак  
sex                - бинарный признак   

Перед обучением нужно будет закодировать категориальные признаки one-hot encoding

### 3. Определить параметры числовых данных

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
df.describe()

Видим, что сильно выделяется признак массы тела, он на 2-3 порядка больше остальных признаков. Перед обучением было бы неплохо провести нормализацию

### 4. Для нечисловых (текстовых) данных определить количество записей по каждому элементу и визуализировать гистограммы, например, так (для признака «species»)

In [ ]:
category_columns = ['species', 'island', 'sex']

fig, ax = plt.subplots(1, 3, figsize=(12, 4))
plt.subplots_adjust(wspace=0.5)

for i, category in enumerate(category_columns):
    sns.countplot(data=df, x=category, ax=ax[i])
    ax[i].set_title(f'Признак {category}')
    ax[i].set_ylabel('Количество')

### 5. Для каждого текстового признака построить мозаику 2D диаграмм рассеяния выбирая в качестве параметров всевозможные пары числовых признаков, аналогично мозаике диаграмм для ирисов.

In [ ]:
pairplot = sns.pairplot(df, hue='species')
pairplot.fig.suptitle('Парные графики пингвинов по признаку вида', y=1.02)

Пингвинов хорошо получиться классифицировать по признаку пола


In [ ]:
ax[i].set_ylabel('Количество')
palette = {'male': 'blue', 'female': 'red'}
pairplot = sns.pairplot(df, hue='sex', palette=palette, )
pairplot.fig.suptitle('Парные графики пингвинов по признаку пола', y=1.02)

Судя по графикам удастся хорошо классифицировать пингвинов по классам.

In [ ]:
pairplot = sns.pairplot(df, hue='island')
pairplot.fig.suptitle('Парные графики пингвинов по признаку острова', y=1.02)

Видно, что пингвины с одними и теми же массо-габаритными характеристиками живут на разных островах. Классифицировать по этому признаку будет трудно.

## Реализация Персептрона и других вспомогательных функций

In [ ]:
class Perceptron:
  def __init__(self, hidden_layers : List[int],
               activations: List[Callable],
               eta:Union[int, float] = 1,
               n_epochs: int = 100,
               random_weights=None,
               random_state=None):
    np.random.seed(random_state)
    random.seed(random_state)
    self.is_fitted = False
    self.layers = []
    self.epochs = n_epochs
    self.eta = eta
    self.errors = []
    l = hidden_layers + [1]
    for i in range(len(hidden_layers)):
      self.layers.append(Layer((l[i]+1, l[i+1]), activations[i], i + 1, random_weights=random_weights))

  @staticmethod
  def relu(x):
      return np.maximum(0, x)
  @staticmethod
  def step(x):
      return (x >= 0).astype(np.float64)

  @staticmethod
  def sigmoid(x):
    return np.where(x >= 0,
                   1 / (1 + np.exp(-x)),
                   np.exp(x) / (1 + np.exp(x)))

  @staticmethod
  def shuffle(X, y):
    n = len(y)
    a = [(random.random(), X[i, :], y[i]) for i in range(n)]
    a.sort()
    new_X = np.array([a[i][1] for i in range(n)])
    new_y = np.array([a[i][2] for i in range(n)])
    return new_X, new_y

  @staticmethod
  def get_grad(activation):
      activation = activation.pyfunc
      if activation == Perceptron.sigmoid:
          s = Perceptron.sigmoid
          return lambda x: s(x) * (1 - s(x))
      return lambda x: (x >= 0).astype(np.int64)


  def predict(self, train_sample: NDArray, logging: bool = False) -> NDArray:
    result = np.zeros(train_sample.shape[0])
    for i in range(train_sample.shape[0]):
      x = train_sample[i, :]
      for layer in self.layers:
        x = np.append(x, values=[1]) # добавление свободного коэффициента
        x = layer.forward(x, logging)

      result[i] = x[0]
      if logging:
        print(result[i])
    return result

  def train(self, train_sample: NDArray,
          train_ans: NDArray,
          batch_size: int = 32,  # Размер пакета (можно менять)
          logging=False,
          activation=np.sign,
          random_weights=None) -> list[float]:
    if not self.is_fitted:
        self.layers = [Layer((train_sample.shape[1]+1,
                            self.layers[0].size[0]-1),
                            activation,
                            0,
                            random_weights=random_weights)] + self.layers
        self.is_fitted = True

    n_samples = train_sample.shape[0]
    for epoch in range(self.epochs):
        train_sample, train_ans = Perceptron.shuffle(train_sample, train_ans)
        error = 0

        # Разбиваем данные на пакеты
        for batch_start in range(0, n_samples, batch_size):
            batch_end = min(batch_start + batch_size, n_samples)
            batch_X = train_sample[batch_start:batch_end, :]
            batch_y = train_ans[batch_start:batch_end]

            # Инициализируем накопители градиентов для каждого слоя
            weight_updates = [np.zeros_like(layer.w) for layer in self.layers]
            batch_error = 0

            # Обрабатываем каждый пример в пакете
            for i in range(batch_X.shape[0]):
                x = batch_X[i, :]
                target = batch_y[i]

                # Прямое распространение (forward pass)
                activations = [x]
                for layer in self.layers:
                    x = np.append(x, values=[1])
                    x = layer.forward(x, logging)
                    activations.append(x)

                predicted = x[0]
                delta = target - predicted
                batch_error += abs(delta)

                # Обратное распространение (backward pass)
                delta_next = delta
                for layer_idx in range(len(self.layers)-1, -1, -1):
                    layer = self.layers[layer_idx]
                    layer_input = activations[layer_idx]
                    delta_next, weight_update = layer.backprop(delta_next, layer_input, self.eta)
                    weight_updates[layer_idx] += weight_update  # Накопление градиентов

            # Усредняем градиенты и обновляем веса
            for layer, update in zip(self.layers, weight_updates):
                layer.w += (self.eta / batch_size) * update  # Усреднённое обновление

            error += batch_error

        self.errors.append(error)
    return self.errors


class Layer:
  def __init__(self,
               size: tuple[int, int],
               activation: Callable,
               index: int,
               value: Union[int, float]=0,
               random_weights=None):
    self.size = size
    self.w = np.full(size, value, dtype=np.float64)
    if random_weights is not None:
      self.w = np.random.uniform(random_weights[0], random_weights[1], self.size)
    self.activation = np.vectorize(activation)
    self.i = index
    self.last_result = np.array([])
    self.last_x = np.array([])
    self.last_m = np.array([])

  def backward(self, value) -> None:
    if value > 0:
      # надо увеличить те веса, где нет активации,
      # но должна быть активация
      d = (self.last_x>0)*value
      d = np.repeat(np.array([d]).T, self.w.shape[1], axis=1)
      self.w = self.w + d
    if value < 0:
      # надо уменьшить те веса,
      # где активации быть не должно
      d = (self.last_x>0)*value
      d = np.repeat(np.array([d]).T, self.w.shape[1], axis=1)
      self.w = self.w + d
  def backprop(self, delta_prev, layer_input, eta):
    # Получаем взвешенную сумму (последний расчёт в forward)
    m = self.last_m

    # Вычисляем производную активационной функции
    grad = Perceptron.get_grad(self.activation)(m)

    # delta для текущего слоя = delta_prev * производная активации
    delta = delta_prev * grad

    # Добавляем bias-единицу к входному вектору (если её ещё нет)
    layer_input_with_bias = np.append(layer_input, 1)

    # Возвращаем ошибку для предыдущего слоя (без учёта bias-весов)
    delta_next = delta @ self.w[:-1].T

    # Возвращаем градиент для накопления (без обновления весов)
    weight_update = np.outer(layer_input_with_bias, delta)

    return delta_next, weight_update

  def forward(self, x, logging) -> NDArray:
    if logging:
      print(f"Слой №{self.i+1}")
      print(f"Сенсоры  : {x}")
      print(f"Размер: {self.size}")

    m = np.dot(x, self.w)
    result = self.activation(m)
    self.last_result = result
    self.last_x = x
    self.last_m = m
    if logging:
      print(f"Сумматор : {m}")
      print(f"Активация: {result}")
      print(f"Результат размера {result.shape}")

    return result

def count_metrics(predicted : NDArray[bool], real: NDArray[bool], label='') -> None:
    print(f'{label}: ')
    print('Accuracy: ', acc(predicted, real))
    print('Recall: ', rec(predicted, real))
    print('Precision: ', pre(predicted, real))


def get_features(data, target, target_name, features) -> (NDArray, NDArray):

    data = data[features + [target]]
    data.loc[:, target] = (data[target] == target_name)
    data_y = data[[target]].to_numpy().reshape(1, -1)[0] * 1
    data_x = data[features].to_numpy()

    return data_x, data_y


def build_errors(errors: list[float], perceptron: Perceptron) -> None:
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.plot(list(range(len(errors))), errors, marker='o', linestyle='dashed')

    plt.xlabel("Эпохи")
    plt.ylabel("Количество ошибок за эпоху")
    plt.title("Обучение нейросети\nбинарной классификации на числовых признаках")
    print("Размерность слоёв: ", *[i.size for i in  perceptron.layers])

In [ ]:
def print_contour(model, X, y, main_1=0, main_2=1, supp=None, columns_num=None, label = ''):
    # 1. Генерируем сетку для 2-х выбранных признаков
    if supp is None:
        supp = [2, 3]
    if columns_num is None:
        columns_num = [''] * 2
    x_min, x_max = X[:, main_1].min() - 1, X[:, main_1].max() + 1
    y_min, y_max = X[:, main_2].min() - 1, X[:, main_2].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

    # 2. Создаём массив точек в 4-х мерном пространстве, но только для выбранных 2-х признаков
    mean_vals = np.mean(X, axis=0)  # Средние значения для всех 4-х признаков
    fixed_features = mean_vals[supp]  # Средние значения для 3-го и 4-го признаков

    grid_points_4d = np.zeros((xx.ravel().shape[0], 2+len(supp)))
    grid_points_4d[:, [main_1, main_2]] = np.column_stack((xx.ravel(), yy.ravel()))  # 2 выбранных признака
    grid_points_4d[:, [supp]] = fixed_features  # Остальные 2 признака - средние значения

    # 3. Предсказываем классы для сеточных точек в 4-х мерном пространстве
    Z = model.predict(grid_points_4d)
    Z = Z.reshape(xx.shape)

    # 4. Визуализация
    plt.contourf(xx, yy, Z, alpha=0.8, cmap='coolwarm')
    plt.contour(xx, yy, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'], linewidths=[1, 2, 1])

    # Отображаем точки данных
    plt.scatter(X[:, main_1], X[:, main_2], c=y, cmap='coolwarm', edgecolor='k')
    plt.xlabel(columns_num[main_1])
    plt.ylabel(columns_num[main_2])
    plt.title(f'Проекция линий уровня персептрона на 2D {label}')
    plt.show()


def count_metrics(predicted : NDArray[bool], real: NDArray[bool], label='') -> None:
    print(f'{label}: ')
    print('Accuracy: ', acc(predicted, real))
    print('Recall: ', rec(predicted, real))
    print('Precision: ', pre(predicted, real))


def get_features(data, target, target_name, features) -> (NDArray, NDArray):

    data = data[features + [target]]
    data.loc[:, target] = (data[target] == target_name)
    data_y = data[[target]].to_numpy().reshape(1, -1)[0] * 1
    data_x = data[features].to_numpy()

    return data_x, data_y


def build_errors(errors: list[float], perceptron: Perceptron) -> None:
    ax = plt.figure().gca()
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.plot(list(range(len(errors))), errors, marker='o', linestyle='dashed')

    plt.xlabel("Эпохи")
    plt.ylabel("Количество ошибок за эпоху")
    plt.title("Обучение нейросети\nбинарной классификации на числовых признаках")
    print("Размерность слоёв: ", *[i.size for i in  perceptron.layers])



## Задание 2
1. Используя персептрон, постройте бинарный классификатор для определения пола пингвина на основе только числовых признаков. Оцените качество работы классификатора.

2. Постройте 3 классификатора для определения пола пингвина на основе числовых признаков для каждого из видов пингвинов. Оцените их качество. Сравните точности классификаторов между собой и с классификатором п.2.1.

3. Попробуйте каждый из 3-х классификаторов п.2.2 проверить на всей выборке (т.е. на пингвинах разных видов)


### 1. Используя персептрон, постройте бинарный классификатор для определения пола пингвина на основе только числовых признаков. Оцените качество работы классификатора.

Создаим список числовых признаков

In [ ]:
numeric_features = ['bill_length_mm', 'bill_depth_mm', 'body_mass_g', 'flipper_length_mm']


In [ ]:
X, y = get_features(df, 'sex', 'male', numeric_features) #используем функцию для получения необходмых признаков

scaler = StandardScaler() #Стандартизация
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19) # Разбиение в соответствии с замечанием


In [ ]:
X_train

In [ ]:
y_train.sum()

In [ ]:
MLP = Perceptron([300], [lambda x: Perceptron.relu(x).astype(np.int64)], n_epochs=200, random_state=13, eta=0.0001) #Иницализируем персептрон да поинтереснее
errors = MLP.train(np.array(X_train, dtype=np.float64), np.array(y_train, dtype=np.float64) , logging=False, activation=Perceptron.sigmoid, batch_size=50, random_weights=[-3, 3])

In [ ]:
MLP.predict(np.array(X_test, dtype=np.float64))

Наблюдаем неравномерную сходимость к минимуму ошибок

In [ ]:
build_errors(errors, MLP)

Визуально видим как неплохо получилось у персептрона на срезе отделить два признака

In [ ]:
print_contour(MLP, X, y, 0, 1, [2, 3], numeric_features, 'только числовые признаки и бинарная классификация')

In [ ]:
y_test_pred = np.array(MLP.predict(X_test)) == 1
y_train_pred = np.array(MLP.predict(X_train)) == 1

count_metrics(y_train_pred, y_train == 1, 'Обучающая выборка')
count_metrics(y_test_pred,  y_test == 1, 'Тестовая выборка')

Естественно метрика на обучающей выборке 1 так как мы сошлись к минимуму ошибки, а на тестовой метрика ниже, нельзя сказать что значительно ниже, но и незначительной эту разницу не назовешь

### 2. Постройте 3 классификатора для определения пола пингвина на основе числовых признаков для каждого из видов пингвинов. Оцените их качество. Сравните точности классификаторов между собой и с классификатором п.2.1.

In [ ]:
df[df['species'] == 'Adelie'] #смотрим что там вообще лежит

In [ ]:
def get_features(data, target, target_name, features):

    data = data[features + [target]]
    data.loc[:, target] = (data[target] == target_name)
    data_y = data[[target]].to_numpy().reshape(1, -1)[0] * 1
    data_x = data[numeric_features].to_numpy()

    return data_x, data_y


X1, y1, = get_features(df[df['species'] == 'Adelie'], 'sex', 'male', numeric_features) #получаем для всех признаков выборку(можно было здесь в форе но не принципиально)
X2, y2 = get_features(df[df['species'] == 'Gentoo'], 'sex', 'male', numeric_features)
X3, y3 = get_features(df[df['species'] == 'Chinstrap'], 'sex', 'male', numeric_features)

scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()
#стандартизация для всех признаков
species = ['Adelie', 'Gentoo', 'Chinstrap']

X1 = scaler1.fit_transform(X1)
X2 = scaler2.fit_transform(X2)
X3 = scaler3.fit_transform(X3)
scalers = [scaler1, scaler2, scaler3]
datas = [X1, X2, X3]
targets = [y1, y2, y3]

In [ ]:
X_train

тут уже в форе обучаем персептроны для каждого вида отдельно

In [ ]:
percps = []

X_trains = []
X_tests = []
y_trains = []
y_tests = []
#теперь уже в форе обучаем персептроны
for i in range(len(datas)):
    X_local_train, X_local_test, y_local_train, y_local_test = train_test_split(datas[i], targets[i], test_size=0.3, random_state=19)
    percps.append(Perceptron([50], [Perceptron.step], n_epochs=100, random_state=13))
    X_trains.append(X_local_train)
    X_tests.append(X_local_test)
    y_trains.append(y_local_train)
    y_tests.append(y_local_test)
    errors = percps[i].train(X_local_train, y_local_train , logging=False, activation=Perceptron.step, random_weights=[-100, 100])


считаем метрики для каждого классификатора

In [ ]:
for i in range(len(datas)):
    y_train_pred = np.array(percps[i].predict(X_trains[i])) == 1
    y_test_pred = np.array(percps[i].predict(X_tests[i])) == 1

    y_train = y_trains[i] == 1
    y_test  = y_tests[i] == 1
    count_metrics(y_train, y_train_pred, f'Обучающая выборка {species[i]}')
    count_metrics(y_test, y_test_pred, f'Тестовая выборка {species[i]}')


В графики добавлю только вид пингвина так как задача та же бинарная классификация

In [ ]:
build_errors(errors, percps[0], species[0])

In [ ]:
build_errors(errors, percps[1], species[0])

In [ ]:
build_errors(errors, percps[2], species[0])

In [ ]:
print_contour(percps[0], X_trains[0], y_trains[0], 0, 1, [2, 3], numeric_features, species[0])

In [ ]:
print_contour(percps[1], X_trains[1], y_trains[1], 0, 1, [2, 3], numeric_features, species[0])

In [ ]:
print_contour(percps[-1], X_trains[-1], y_trains[-1], 0, 1, [2, 3], numeric_features, species[0])

Понятно что по видам персептрону было проще разделить так как все эти параметры напрямую зависят от вида

### 3. Попробуйте каждый из 3-х классификаторов п.2.2 проверить на всей выборке (т.е. на пингвинах разных видов)

In [ ]:
for i in range(3):
    x_val, y_val = get_features(df, 'sex', 'male', numeric_features)
    x_val = scalers[i].transform(x_val)
    y_val_pred = percps[i].predict(x_val)

    count_metrics(y_val_pred == 1, y_val == 1, 'По всему')

На общей выборке метрики несильно отличаются друг от друга, так как сильно подогнались под виды

## Задание 3
 Бинарная классификация с использованием ADALINE.
1. Повторите пп.1-3 предыдущего задания, построив классификатор на основе адаптивного нейрона.

2. Включите в список признаков вид пингвина, представив его числом (1,2,3) для разных видов. Оцените качество классификации и сравните с результатом пункта 3.1.

3. Замените признак вид на три бинарных признака Adelie, Chinstrap, Gentoo, в каждом из которых значение 1 соответствует тому, что данный пингвин принадлежит соответствующему виду:

4. Выполните классификацию по полу, используя уже 7 числовых признаков. Сравните качество классификации с предыдущими вариантами.

5. Выполните классификацию по каждому из видов пингвина, используя 5 числовых признаков (4 размера-веса и пол: 1-male, 0-female). Сравните качество классификаторов.

6. Повторите предыдущий пункт, добавив к 5 числовым признакам ещё три бинарных, представляющих принадлежность к тому или иному острову.


### 1. Повторите пп.1-3 предыдущего задания, построив классификатор на основе адаптивного нейрона.

Просто повторяем задание 2 полностью но персептрон используем тот же

In [ ]:


X, y = get_features(df, 'sex', 'male', numeric_features)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)

adaline = Perceptron([1000], [Perceptron.step], n_epochs=50, random_state=13)
errors = adaline.train(X_train, y_train, logging=False, activation=Perceptron.step, random_weights=[-100, 100])

build_errors(errors, adaline)


In [ ]:
print_contour(adaline, X, y, 0, 1, [2, 3], numeric_features)

In [ ]:

copy_df = df.copy()

X1, y1, = get_features(copy_df[copy_df['species'] == 'Adelie'], 'sex', 'male', numeric_features)
X2, y2 = get_features(copy_df[copy_df['species'] == 'Gentoo'], 'sex', 'male', numeric_features)
X3, y3 = get_features(copy_df[copy_df['species'] == 'Chinstrap'], 'sex', 'male', numeric_features)

scaler = StandardScaler()

X1 = scaler.fit_transform(X1)
X2 = scaler.transform(X2)
X3 = scaler.transform(X3)

datas = [X1, X2, X3]
targets = [y1, y2, y3]


X_trains = []
X_tests = []
y_trains = []
y_tests = []
for i in range(len(datas)):
    X_local_train, X_local_test, y_local_train, y_local_test = train_test_split(datas[i], targets[i], test_size=0.3,
                                                                                random_state=19)
    percps.append(Perceptron([50], [Perceptron.step], n_epochs=100, random_state=13))
    X_trains.append(X_local_train)
    X_tests.append(X_local_test)
    y_trains.append(y_local_train)
    y_tests.append(y_local_test)
    errors = adaline.train(X_local_train, y_local_train, logging=False, activation=Perceptron.step,
                             random_weights=[-100, 100])

for i in range(len(datas)):
    y_train_pred = np.array(adaline.predict(X_trains[i])) == 1
    y_test_pred = np.array(adaline.predict(X_tests[i])) == 1

    y_train = y_trains[i] == 1
    y_test = y_tests[i] == 1

    count_metrics(y_train_pred, y_train, 'Обучающая')
    count_metrics(y_test_pred, y_test, 'Тестовая')
X, y = get_features(copy_df, 'sex', 'male', numeric_features)
X = scaler.transform(X)


y_test_pred = adaline.predict(X) == 1
y_test = y == 1
count_metrics(y_test_pred, y_test, 'ALL_X')

In [ ]:
build_errors(errors, adaline, 'дообученная на разных выборках')

In [ ]:
print_contour(adaline, X, y, 0, 1, [2, 3], numeric_features, 'дообученный на разных выборках')

После повторения всех шагов без комментариев так как они есть в предыдущем пункте, мы получили метрику значительно хуже чем в персептроне неадаптивном, то есть лучше не дообучать пока нет обратного распротсранения ошибки через градиенты

### 2. Включите в список признаков вид пингвина, представив его числом (1,2,3) для разных видов. Оцените качество классификации и сравните с результатом пункта 3.1.

In [ ]:
def get_features(data, target, target_name, features) -> (NDArray, NDArray):

    data = data[features + [target]]
    data.loc[:, target] = (data[target] == target_name)
    data_y = data[[target]].to_numpy().reshape(1, -1)[0] * 1
    data_x = data[features].to_numpy()

    return data_x, data_y

adaline_cat = Perceptron([1000], [Perceptron.step], n_epochs=100, random_state=13, eta=0.001)
new_features = numeric_features + ['species']
copy_df = df.copy()

labelEncoder = LabelEncoder()
scaler = StandardScaler()
copy_df['species'] = labelEncoder.fit_transform(copy_df['species']) + 1
X, y = get_features(copy_df, 'sex', 'male', numeric_features + ['species'])
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)





In [ ]:
errors = adaline_cat.train(X_train, y_train, logging=False, activation=Perceptron.step, random_weights=[-100, 100])

In [ ]:
X_train

In [ ]:
y_train_pred = np.array(adaline_cat.predict(X_train)) == 1
y_test_pred = np.array(adaline_cat.predict(X_test)) == 1
y_train = y_train == 1
y_test = y_test == 1
count_metrics(y_train_pred, y_train, 'Обучающая выборка')
count_metrics(y_test_pred, y_test, 'Тестовая выборка')

Метрика получилась почти такая же как и на без вида, скорее всего потому что мы использовали вид как число

In [ ]:

build_errors(errors, adaline_cat)



In [ ]:
print_contour(adaline_cat, X, y, 0, 1, [2, 3, 4], new_features)

### 3 Замените признак вид на три бинарных признака Adelie, Chinstrap, Gentoo, в каждом из которых значение 1 соответствует тому, что данный пингвин принадлежит соответствующему виду:


In [ ]:
adaline_one_hot = Perceptron([1000], [Perceptron.step], n_epochs=100, random_state=13)
new_features = numeric_features + ['species']


ohe = OneHotEncoder(sparse_output=False)
ohe.set_output(transform='pandas')
copy_df = df.copy()


df_species = ohe.fit_transform(df[['species']].to_numpy())

copy_df = pd.concat([copy_df[numeric_features], df_species], axis=1)
copy_df




In [ ]:
temp_df = pd.concat([copy_df, df[['sex']]], axis=1)


Разбиваем на выборки тестовую и обучающую

In [ ]:

X, y = get_features( temp_df, 'sex', 'male', numeric_features + list(ohe.get_feature_names_out()))
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)


### 4. Выполните классификацию по полу, используя уже 7 числовых признаков. Сравните качество классификации с предыдущими вариантами.

In [ ]:
X_train #смотрим что тут есть

In [ ]:
errors = adaline_one_hot.train(X_train, y_train, logging=False, activation=Perceptron.step, random_weights=[-100, 100])
y_train_pred = np.array(adaline_one_hot.predict(X_train)) == 1
y_test_pred = np.array(adaline_one_hot.predict(X_test)) == 1
y_train = y_train == 1
y_test = y_test == 1
count_metrics(y_train_pred, y_train, 'Обучающая выборка')
count_metrics(y_test_pred, y_test, 'Тестовая выборка')

Видим что метрика улучшилась на тестовой но не сильно можно списать это на везение при разбиении хотя использовали тот же random_state

In [ ]:
build_errors(errors, adaline_one_hot)

In [ ]:
print_contour(adaline_one_hot, X, y, 0, 1, [2, 3, 4, 5, 6], new_features)

### 5. Выполните классификацию по каждому из видов пингвина, используя 5 числовых признаков (4 размера-веса и пол: 1-male, 0-female). Сравните качество классификаторов.

In [ ]:
copy_df['sex'] = (df['sex'] == 'male').apply(float)

In [ ]:
copy_df

In [ ]:
X_adelie, y_adelie = get_features(copy_df, 'x0_Adelie', True, numeric_features + ['sex'])
X_chinstrap, y_chinstrap = get_features(copy_df, 'x0_Chinstrap', True, numeric_features + ['sex'])
X_gentoo, y_gentoo = get_features(copy_df, 'x0_Gentoo', True, numeric_features + ['sex'])
scaler = StandardScaler()
X_adelie = scaler.fit_transform(X_adelie)
X_chinstrap = scaler.transform(X_chinstrap)
X_gentoo = scaler.transform(X_gentoo)

In [ ]:
X_adelie

In [ ]:
X_penguins = [X_adelie, X_chinstrap, X_gentoo]
y_penguins = [y_adelie, y_chinstrap, y_gentoo]
X_trains = []
X_tests = []
y_trains = []
y_tests = []
percps_pen = []
for i in range(len(datas)):
    X_local_train, X_local_test, y_local_train, y_local_test = train_test_split(X_penguins[i], y_penguins[i], test_size=0.3,
                                                                                random_state=19)
    percps_pen.append(Perceptron([50], [Perceptron.step], n_epochs=100, random_state=13))
    X_trains.append(X_local_train)
    X_tests.append(X_local_test)
    y_trains.append(y_local_train)
    y_tests.append(y_local_test)
    errors = percps_pen[i].train(X_local_train, y_local_train, logging=False, activation=Perceptron.step,
                             random_weights=[-100, 100])

for i in range(len(datas)):
    y_train_pred = np.array(percps_pen[i].predict(X_trains[i])) == 1
    y_test_pred = np.array(percps_pen[i].predict(X_tests[i])) == 1

    y_train = y_trains[i] == 1
    y_test = y_tests[i] == 1

    count_metrics(y_train_pred, y_train, 'Обучающая')
    count_metrics(y_test_pred, y_test, 'Тестовая')



In [ ]:
build_errors(errors, percps_pen[0], species[0])

In [ ]:
build_errors(errors, percps_pen[1], species[1])

In [ ]:
build_errors(errors, percps_pen[2], species[2])

Все сети сошлись на первой эпохе, очень удивительно пять раз проверил вроде таргет не закинули в обучающую

In [ ]:
print_contour(percps_pen[0], X_penguins[0], y_penguins[0], 0, 1, [2, 3, 4], new_features, species[0])

In [ ]:
print_contour(percps_pen[1], X_penguins[1], y_penguins[1], 0, 1, [2, 3, 4], new_features, species[1])

In [ ]:
print_contour(percps_pen[2], X_penguins[2], y_penguins[2], 0, 1, [2, 3, 4], new_features, species[2])

Плюсом по графикам видно что он справился с разделением

### 6. Повторите предыдущий пункт, добавив к 5 числовым признакам ещё три бинарных, представляющих принадлежность к тому или иному острову.

In [ ]:
ohe_islands = OneHotEncoder(sparse_output=False)
ohe.set_output(transform='pandas')
df_islands = ohe.fit_transform(df[['island']].to_numpy())

copy_df = pd.concat([copy_df, df_islands], axis=1)
copy_df


In [ ]:
X_adelie, y_adelie = get_features(copy_df, 'x0_Adelie', True, numeric_features + ['sex'] + ['x0_Biscoe', 'x0_Dream', 'x0_Torgersen'])
X_chinstrap, y_chinstrap = get_features(copy_df, 'x0_Chinstrap', True, numeric_features + ['sex'] + ['x0_Biscoe', 'x0_Dream', 'x0_Torgersen'])
X_gentoo, y_gentoo = get_features(copy_df, 'x0_Gentoo', True, numeric_features + ['sex'] + ['x0_Biscoe', 'x0_Dream', 'x0_Torgersen'])
scaler = StandardScaler()
X_adelie = scaler.fit_transform(X_adelie)
X_chinstrap = scaler.transform(X_chinstrap)
X_gentoo = scaler.transform(X_gentoo)

In [ ]:
X_adelie #смотрим что внутри

In [ ]:
X_penguins = [X_adelie, X_chinstrap, X_gentoo]
y_penguins = [y_adelie, y_chinstrap, y_gentoo]
X_trains = []
X_tests = []
y_trains = []
y_tests = []
percps_pen = []
for i in range(len(datas)):
    X_local_train, X_local_test, y_local_train, y_local_test = train_test_split(X_penguins[i], y_penguins[i], test_size=0.3,
                                                                                random_state=19)
    percps_pen.append(Perceptron([50], [Perceptron.step], n_epochs=100, random_state=13))
    X_trains.append(X_local_train)
    X_tests.append(X_local_test)
    y_trains.append(y_local_train)
    y_tests.append(y_local_test)
    errors = percps_pen[i].train(X_local_train, y_local_train, logging=False, activation=Perceptron.step,
                             random_weights=[-100, 100])

for i in range(len(datas)):
    y_train_pred = np.array(percps_pen[i].predict(X_trains[i])) == 1
    y_test_pred = np.array(percps_pen[i].predict(X_tests[i])) == 1

    y_train = y_trains[i] == 1
    y_test = y_tests[i] == 1

    count_metrics(y_train_pred, y_train, 'Обучающая')
    count_metrics(y_test_pred, y_test, 'Тестовая')

In [ ]:
build_errors(errors, percps_pen[0], species[0])

In [ ]:
build_errors(errors, percps_pen[1], species[1])

In [ ]:
build_errors(errors, percps_pen[2], species[2])

In [ ]:
print_contour(percps_pen[0], X_penguins[0], y_penguins[0], 0, 1, [2, 3, 4, 5, 6, 7], new_features, species[0])

In [ ]:
print_contour(percps_pen[1], X_penguins[1], y_penguins[1], 0, 1, [2, 3, 4, 5, 6, 7], new_features, species[1])

In [ ]:
print_contour(percps_pen[2], X_penguins[2], y_penguins[2], 0, 1, [2, 3, 4,5, 6, 7], new_features, species[2])

Метрика не улучшилась по очевидным причинам лучше некуда, и не ухудшилась, получается остров лишнее для определение вида

## Задание 4
Множественная классификация с использованием ADALINE
1. Постройте сеть из 4 нейронов. На первом слое три из них определяют принадлежность к одному из трёх видов пингвинов. На втором слое на основе оценок 1-го слоя выносится суждение о том, какому из видов принадлежит конкретный объект. Нейрон этого слоя тоже надо обучить.
Провести расчёты в трёх вариантах:
2. используя 4 числовых слоя (только размеры-веса)
3. используя 5 числовых слоёв (+пол)
4. используя 8 числовых слоёв (+остров).
5. Оценить качество полученных классификаторов. Сделать вывод.

Оценка за работу 5 баллов. Мало(
Дополнительные 2 балла могут быть выставлены, если обучение каждого классификатора сопровождается выводом графика динамики качества обучения по эпохам и диаграммы классификации областей (оси – два числовых признака размер/вес на выбор студента)
Дополнительный 1 балл за вывод перед каждым обучением информации о характере обучения и первых 5-ти строк обучающей выборки.
 Доп задания сделаны.


### 1. Постройте сеть из 4 нейронов. На первом слое три из них определяют принадлежность к одному из трёх видов пингвинов. На втором слое на основе оценок 1-го слоя выносится суждение о том, какому из видов принадлежит конкретный объект. Нейрон этого слоя тоже надо обучить.

### 2. используя 4 числовых слоя (только размеры-веса)

формируем выборку исходя из потребностей

In [ ]:
copy_df = df.copy()

new_features = numeric_features + ['species']
copy_df = df.copy()

labelEncoder = LabelEncoder()
scaler = StandardScaler()
copy_df['sex'] = (copy_df['sex'] == 'male').apply(float)
copy_df['species'] = labelEncoder.fit_transform(copy_df['species'])
X, _ = get_features(copy_df, 'species', 'Adelie', numeric_features )
y = copy_df['species'].to_numpy().reshape(1, -1)[0] * 1
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)



In [ ]:
X_train #убеждаемся в соответствии

In [ ]:
MLP_targ = Perceptron([3], [lambda x: round(min(Perceptron.relu(x), 2))], n_epochs=200, random_state=16, eta=0.0001)
errors = MLP_targ.train(X_train, y_train, logging=False, activation=Perceptron.relu,
                             random_weights=[-1, 1])


In [ ]:
build_errors(errors, MLP_targ)
plt.ylim((1, 250))

In [ ]:
print_contour(MLP_targ, X, y, 0, 1, [2, 3], new_features)

In [ ]:
MLP_targ.predict(X_train)

In [ ]:
acc(MLP_targ.predict(X_train), y_train)

In [ ]:
acc(MLP_targ.predict(X_test), y_test)

Метрики мягко говоря не очень, но хотя бы не наивный классификатор

### 3. используя 5 числовых слоёв (+пол)

In [ ]:
copy_df = df.copy()

new_features = numeric_features + ['species']
copy_df = df.copy()

labelEncoder = LabelEncoder()
scaler = StandardScaler()
copy_df['sex'] = (copy_df['sex'] == 'male').apply(float)
copy_df['species'] = labelEncoder.fit_transform(copy_df['species'])
X, _ = get_features(copy_df, 'species', 'Adelie', numeric_features +['sex'])
y = copy_df['species'].to_numpy().reshape(1, -1)[0] * 1
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)



In [ ]:
X_train #убеждаемся в том что пол появился

In [ ]:
MLP_targ = Perceptron([3], [lambda x: round(min(Perceptron.relu(x), 2))], n_epochs=200, random_state=16, eta=0.0001)
errors = MLP_targ.train(X_train, y_train, logging=False, activation=Perceptron.relu,
                        random_weights=[-1, 1])


In [ ]:
build_errors(errors, MLP_targ, 'Многоклассовой классификации')

In [ ]:
print_contour(MLP_targ, X, y, 0, 1, [2, 3, 4], new_features, 'Многоклассивоая классификация')

In [ ]:
acc(MLP_targ.predict(X_train), y_train)

In [ ]:
acc(MLP_targ.predict(X_test), y_test)

Сейчас метрики получились ВО! Мы ничего не делали кроме как стандартизировали данные и придумали классную функцию активиации для классификатора остальное за нас сделал персептрон

### 4. используя 8 числовых слоёв (+остров).

In [ ]:
copy_df = df.copy()

new_features = numeric_features + ['species']
copy_df = df.copy()

labelEncoder = LabelEncoder()
scaler = StandardScaler()
copy_df['sex'] = (copy_df['sex'] == 'male').apply(float)


df_islands = ohe.fit_transform(df[['island']].to_numpy())

copy_df = pd.concat([copy_df, df_islands], axis=1)

copy_df['species'] = labelEncoder.fit_transform(copy_df['species'])
X, _ = get_features(copy_df, 'species', 'Adelie', numeric_features +['sex'] + ['x0_Biscoe', 'x0_Dream', 'x0_Torgersen'])
y = copy_df['species'].to_numpy().reshape(1, -1)[0] * 1
X = scaler.fit_transform(X)




X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=19)

In [ ]:
MLP_targ = Perceptron([3], [lambda x: round(min(Perceptron.relu(x), 2))], n_epochs=200, random_state=16, eta=0.0001)
errors = MLP_targ.train(X_train, y_train, logging=False, activation=Perceptron.relu,
                        random_weights=[-1, 1])

In [ ]:
build_errors(errors, MLP_targ, 'Многоклассивоая классификация')

In [ ]:
print_contour(MLP_targ, X, y, 0, 1, [2, 3, 4, 5, 6, 7], new_features, 'Многоклассивоая классификация')

In [ ]:
acc(MLP_targ.predict(X_train), y_train)

In [ ]:
acc(MLP_targ.predict(X_test), y_test)

Метрики улучшились уже незначитель ну потому что уже некуда расти но видно что свое влияние остров оказывает

### Дополнительно глянем на 500 нейронах на первом слое

In [ ]:
MLP_targ = Perceptron([500], [lambda x: round(min(Perceptron.relu(x), 2))], n_epochs=200, random_state=16, eta=0.0001)
errors = MLP_targ.train(X_train, y_train, logging=False, activation=Perceptron.relu,
                        random_weights=[-1, 1])

In [ ]:
build_errors(errors, MLP_targ, 'Многоклассовая классификация')

In [ ]:
print_contour(MLP_targ, X, y, 0, 1, [2, 3, 4, 5, 6, 7], new_features, 'Многоклассовая классификация 500 нейронов на первом слое')

In [ ]:
acc(MLP_targ.predict(X_train), y_train)

In [ ]:
acc(MLP_targ.predict(X_test), y_test)

Глянули что если поставить больше слоев, видно что нейросеть как-то пытается разделить эти данные, и у нее неплохо получается метрика 0.97

## Делаем выводы

Сходимость у персептрона неравномерная, обязательно данные приводить к одному виду и функцию активации нужно использовать relu чтобы сходилось лучше или другие ее подвиды.

Также можно сказать что персептрон хоть и простая нейросеть но все равно очень мощно решает задачи регрессии и классификации даже в самом простом своем виде без обратного распространения ошибки, а математика проще чем в SVM или градиентном спуске, но при этом он универсален, однако за использование правила хебба приходится платить определенную цену в виде неравномерной сходимости и сильной зависимостью от порядка данных

Так же стоит еще посмотреть на категориальные данные есть мысль что персептрон несильно зависит от их математического вида (матрица 0 и 1 либо вектор чисел) есть ощущение что с точки зрения персептрона можно добавить нейронов и он выдаст ту же точность.

Также некоторые задания были не очень понятны потому мы их сделали на свое усмотрение например четвертое задание звучит будто сделайте 3 персептрона и 1 нейрон или как имелось в виду ведь мы уже написали персептрон со скрытыми слоями зачем отдельно обучать другой нейрон, можно же в рамках одного персептрона.

Не очень понятно что такое характер обучения, такое нигде никогда не обсуждалось и дискуссия с одногруппниками ни к чему не привела

Еще было много одинаковых заданий из-за чего пришлось помучаться если бы они были более объемные то можно было бы написать функцию универсальную а тут маленькие повторяющиеся из-за этого как-то неприятно

В целом было очень интересная лабораторная мы справились с многоклассовой классификацией можно попробовать с числами сделать классификацию